In [4]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import time
from datetime import datetime, timedelta
import os.path
import pandas as pd

import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer 


from gensim.summarization.summarizer import summarize
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore
from gensim.test.utils import datapath
from gensim.models import *
from gensim import similarities
from gensim.models import CoherenceModel
from gensim.matutils import softcossim


from GoogleNews import GoogleNews
googlenews = GoogleNews()

import newspaper
from newspaper import Article
from newspaper import fulltext

import pyLDAvis.gensim

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import random
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import docx
from docx import Document
from docx.shared import RGBColor

from parsers import Google_parser as gp


## запрос по api
import requests
import json
from newsapi import NewsApiClient

import gensim.downloader as api
# Download the models
#fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
word2vec_model300 = api.load('word2vec-google-news-300')
#glove_model300 = api.load('glove-wiki-gigaword-300')

from emailing import sendmailto

# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')


In [2]:
#Блок предобработки текста

stop_words = stopwords.words('english')
lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return words #' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

def add_hyperlink(paragraph, url, text):
    """
    A function that places a hyperlink within a paragraph object.
    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

In [3]:
# слова-исключения, по которым будут отфильтровываться статьи
exclude_list = ['macbook','laptop','ultrabook','server', 'api', 'usb', 'installer', 'ubuntu', 'ip', 'lan','javascript', 'captcha', 'browser','internet', 'usb-c']  

### Блок транспортных новостей


In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')


# transport_topiclist = [ 'maritime AND transport', 'port AND freight', '+cargo', 'transport AND delay', 'container AND ship', '+bulk', 'bulk AND carrier',
#                        '+tanker', '+TEU', 'auto AND industry', '+freight', 'refrigerated AND ship', '+reefer', '+supertanker', 'cargo AND ship']

transport_topiclist = [ '+maritime +transport', '+maritime +transport', '+cargo', '+container +ship', '+bulk', '+bulk +carrier',
                       '+tanker', '+TEU', '+auto +industry', '+freight', '+refrigerated +ship', '+reefer', '+supertanker', '+cargo +ship']
transport_all_articles = []
body_fulltext = []

for topic in transport_topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    all_articles = newsapi.get_everything(q=request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=3)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(all_articles['articles'])):
        url  = all_articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            all_articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    transport_all_articles.append(all_articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
transport_all_contents = []
transport_all_titles = []
transport_all_urls = []
for article_list in transport_all_articles:
    i = 0
    for item in article_list['articles']:
        transport_all_contents.append(item['content'])
        transport_all_titles.append(item['title'])
        transport_all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(transport_all_contents)):
    try:
        transport_all_contents[i] = summarize(text = transport_all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in transport_all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
    
# text_0 = mydict.doc2bow(tokenized_list[22])
# text_1 = mydict.doc2bow(tokenized_list[32])
     


# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    comparison_index = 0
    similarity_flag = 0
    for comparison_index in range(text_index+1, len(bag_of_words)):
    #for comparison in bag_of_words:
        #if text_index != comparison_index:
        similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
        if similarity_ratio > 0.2:
            similarity_flag = 1

    if (similarity_flag == 0) and (transport_all_contents[text_index] not in unique_articles):
        #текст потенциально подходит. проверим на наличие перекрестных тем
        #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
        exclude_count = 0
        tokens = prepare_text(transport_all_contents[text_index])
        for word in tokens:
            if word in exclude_list:
                exclude_count += 1
        if exclude_count == 0:
            unique_articles.append(transport_all_contents[text_index])
            unique_urls.append(transport_all_urls[text_index])
            unique_titles.append(transport_all_titles[text_index])
        else:
            pass

        #comparison_index += 1
    #text_index += 1
                
        
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости транспорта'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Ссылка на источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name = 'transport_' + str(current_date) + '.docx'
document.save(doc_name)       

In [ ]:
for k,v in mydict.token2id.items():
    print(k,v)

In [19]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')


# transport_topiclist = [ 'maritime AND transport', 'port AND freight', '+cargo', 'transport AND delay', 'container AND ship', '+bulk', 'bulk AND carrier',
#                        '+tanker', '+TEU', 'auto AND industry', '+freight', 'refrigerated AND ship', '+reefer', '+supertanker', 'cargo AND ship']

transport_topiclist = [ 'cargo freight']
transport_all_articles = []
body_fulltext = []

for topic in transport_topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    all_articles = newsapi.get_everything(q=request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=10)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(all_articles['articles'])):
        url  = all_articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            all_articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    transport_all_articles.append(all_articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
transport_all_contents = []
transport_all_titles = []
transport_all_urls = []
for article_list in transport_all_articles:
    i = 0
    for item in article_list['articles']:
        transport_all_contents.append(item['content'])
        transport_all_titles.append(item['title'])
        transport_all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(transport_all_contents)):
    try:
        transport_all_contents[i] = summarize(text = transport_all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in transport_all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]


--- Выгружаются новости... ---
5
10
--- Завершено. На выгрузку затрачено 6.489573955535889 секунд ---


In [4]:
#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return    words#' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

tokenized_list = []
for doc in transport_all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1


In [ ]:
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

In [ ]:
from gensim.models import TfidfModel
model = TfidfModel(mycorpus)  # fit model
vector = model[mycorpus[0]]
corpus_tfidf = model[mycorpus] #apply transformation to the whole corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(tokenized_list)

In [17]:
len(transport_all_contents)

3

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

def cosine_sim(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [39]:
print(transport_all_contents[0])
print('\n')
print(transport_all_contents[2])

Boeing’s BA 777 freighter would gain the most unless Airbus sees the prospect of orders from blue-chip Chinese airlines as motivation to develop an A350 freighter and break Boeing’s monopoly on new long-range freight aircraft.
Shanghai is the world’s third-largest air cargo hub but local airline China Eastern has a cargo division with only eight freighters.
If carriers want to increase freight aircraft, new sales are likely since converting old passenger aircraft to freighters is seen as inefficient for regular long-haul use.
MORE FROM FORBES These Airlines Will Next Remove Seats To Carry Cargo And Medical Supplies On Passenger Aircraft Turned Into Freighters


SEATTLE, April 29, 2020 /PRNewswire/ -- Alaska Air Cargo announced it will fly passenger jets as cargo-only flights to carry essential goods like mail, medical equipment, e-commerce packages and food throughout its domestic network.
we serve: 19 stations, only three connect by road Around 60% of our cargo business touches the st

In [42]:
cosine_sim(transport_all_contents[0], ['transport'])

AttributeError: 'list' object has no attribute 'lower'

In [8]:
vect=TfidfVectorizer(max_df=0.85,stop_words=stopwords)
tfidf=vect.fit_transform(tokenized_list)

TypeError: 'WordListCorpusReader' object is not iterable

### Блок Агро новостей


In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

topiclist = ['+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
            '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', 
             '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el AND +nino', '+usa AND +drought']
# topiclist = ['+china AND red AND line', '+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
#             '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', '+brazil AND +inflation', '+brazil AND +real', 
#              '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el +nino', '+usa AND +drought']
# topiclist = ['china AND land', 'china AND red AND line', 'agro AND subsidy', 'china AND gmo', 'food AND reserve', 'agriculture AND policy', 'agriculture AND tax',
#             'china AND tax', 'food AND trade', 'india AND fertilizer', 'india AND subsidy', 'china AND subsidy', 'monsoon', 'brazil AND inflation', 'brazil AND real', 
#              'argentina AND tariff', 'argentina AND export', 'EU AND quota', 'eu AND food', 'el nino', 'usa AND drought']
all_articles = []
body_fulltext = []

for topic in topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    articles = newsapi.get_everything(q = request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=3)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(articles['articles'])):
        url  = articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    all_articles.append(articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
all_contents = []
all_titles = []
all_urls = []
for article_list in all_articles:
    i = 0
    for item in article_list['articles']:
        all_contents.append(item['content'])
        all_titles.append(item['title'])
        all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(all_contents)):
    try:
        all_contents[i] = summarize(text = all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
          


# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    comparison_index = 0
    similarity_flag = 0
    for comparison_index in range(text_index+1, len(bag_of_words)):
    #for comparison in bag_of_words:
        #if text_index != comparison_index:
        similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
        if similarity_ratio > 0.2:
            similarity_flag = 1

    if (similarity_flag == 0) and (all_contents[text_index] not in unique_articles):
        #текст потенциально подходит. проверим на наличие перекрестных тем
        #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
        exclude_count = 0
        tokens = prepare_text(all_contents[text_index])
        for word in tokens:
            if word in exclude_list:
                exclude_count += 1
        if exclude_count == 0:
            unique_articles.append(all_contents[text_index])
            unique_urls.append(all_urls[text_index])
            unique_titles.append(all_titles[text_index])
        else:
            pass

        #comparison_index += 1
    #text_index += 1
                
        
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости агросектора'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Ссылка на источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name = 'agro_' + str(current_date) + '.docx'
document.save(doc_name)       

### Energy

In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

topiclist = ['+energy AND +belarus', '+energy AND +kazakhstan', '+energy AND +uzbekistan','+renewable AND +energy' ]
# topiclist = ['+china AND red AND line', '+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
#             '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', '+brazil AND +inflation', '+brazil AND +real', 
#              '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el +nino', '+usa AND +drought']
# topiclist = ['china AND land', 'china AND red AND line', 'agro AND subsidy', 'china AND gmo', 'food AND reserve', 'agriculture AND policy', 'agriculture AND tax',
#             'china AND tax', 'food AND trade', 'india AND fertilizer', 'india AND subsidy', 'china AND subsidy', 'monsoon', 'brazil AND inflation', 'brazil AND real', 
#              'argentina AND tariff', 'argentina AND export', 'EU AND quota', 'eu AND food', 'el nino', 'usa AND drought']
all_articles = []
body_fulltext = []

for topic in topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    articles = newsapi.get_everything(q = request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=3)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(articles['articles'])):
        url  = articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    all_articles.append(articles)
#     #сохранение новостей в базу

#     save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
#     new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
#     with open(new_filename, 'w', encoding='utf-8') as f:
#         json.dump(all_articles, f, ensure_ascii=False, indent=4)
#         print('Выгрузка новостей сохранена в файл')
    

#здесь создаем три отдельных списка для текстов новостей, заголовков, ссылок. заполняем их
all_contents = []
all_titles = []
all_urls = []
for article_list in all_articles:
    i = 0
    for item in article_list['articles']:
        all_contents.append(item['content'])
        all_titles.append(item['title'])
        all_urls.append(item['url'])

# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(all_contents)):
    try:
        all_contents[i] = summarize(text = all_contents[i], word_count = 100)
    except:
        pass

#формируем список списков из предобработанных саммари, разделенных на токены
#при формировании будем проверять наличие слов-исключений, связанных с посторонними темами.
error_index = 0
error_list = []
tokenized_list = []
for doc in all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
          


# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    comparison_index = 0
    similarity_flag = 0
    for comparison_index in range(text_index+1, len(bag_of_words)):
    #for comparison in bag_of_words:
        #if text_index != comparison_index:
        similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
        if similarity_ratio > 0.2:
            similarity_flag = 1

    if (similarity_flag == 0) and (all_contents[text_index] not in unique_articles):
        #текст потенциально подходит. проверим на наличие перекрестных тем
        #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
        exclude_count = 0
        tokens = prepare_text(all_contents[text_index])
        for word in tokens:
            if word in exclude_list:
                exclude_count += 1
        if exclude_count == 0:
            unique_articles.append(all_contents[text_index])
            unique_urls.append(all_urls[text_index])
            unique_titles.append(all_titles[text_index])
        else:
            pass

        #comparison_index += 1
    #text_index += 1
                
        
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости агросектора'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Ссылка на источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name = 'agro_' + str(current_date) + '.docx'
document.save(doc_name)       

In [ ]:
# задаем клиентский апи
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')

# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

topiclist = ['+energy AND +belarus', '+energy AND +kazakhstan', '+energy AND +uzbekistan','+renewable AND +energy' ]
# topiclist = ['+china AND red AND line', '+agriculture AND +subsidy', '+china AND +gmo', '+agriculture AND +policy', '+agriculture AND +tax',
#             '+food AND +trade', '+india AND +fertilizer', '+india AND +subsidy', '+china AND +subsidy', '+monsoon', '+brazil AND +inflation', '+brazil AND +real', 
#              '+argentina AND +tariff', '+argentina AND +export', '+EU AND +quota', '+el +nino', '+usa AND +drought']
# topiclist = ['china AND land', 'china AND red AND line', 'agro AND subsidy', 'china AND gmo', 'food AND reserve', 'agriculture AND policy', 'agriculture AND tax',
#             'china AND tax', 'food AND trade', 'india AND fertilizer', 'india AND subsidy', 'china AND subsidy', 'monsoon', 'brazil AND inflation', 'brazil AND real', 
#              'argentina AND tariff', 'argentina AND export', 'EU AND quota', 'eu AND food', 'el nino', 'usa AND drought']
all_articles = []
body_fulltext = []

for topic in topiclist:
    #задаем тему запроса
    request_topic = topic

    # получение данных по теме
    articles = newsapi.get_everything(q = request_topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=5)
    
    #Формирование служебных переменных
    count = 0
    error_link = [] #массив с битыми ссылками
    


    #замеряем время
    start_time = time.time()
    print('--- Выгружаются новости... ---')

    #попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
    for i in range(0, len(articles['articles'])):
        url  = articles['articles'][i]['url']
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body_fulltext.append(text) 
            articles['articles'][i]['content'] = text 
        except:
            error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)
    
    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))
    all_articles.append(articles)

In [ ]:
all_articles

In [ ]:
tfidf = models.TfidfModel(mycorpus, smartirs='ntc')
for doc in tfidf[mycorpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

In [ ]:


# Prepare the similarity matrix
similarity_matrix = word2vec_model300.similarity_matrix(mydict, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

In [ ]:
# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))
    
# text_0 = mydict.doc2bow(tokenized_list[22])
# text_1 = mydict.doc2bow(tokenized_list[32])


In [ ]:
# слова-исключения, по которым будут отфильтровываться статьи
exclude_list = ['macbook','laptop','ultrabook','server', 'api', 'usb', 'installer', 'ubuntu', 'ip', 'lan','javascript', 'captcha', 'browser','internet', 'usb-c']        


In [ ]:
# производится проверка на наличие схожих текстов. 
# если степень схожется меньше 0.5 проверяем на наличие текста в списке уникальных
# добавляем в список уникальных при условии несхожести и отсутствия дублирования
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    comparison_index = 0
    similarity_flag = 0
    for comparison_index in range(text_index+1, len(bag_of_words)):
    #for comparison in bag_of_words:
        #if text_index != comparison_index:
        similarity_ratio = softcossim(bag_of_words[text_index], bag_of_words[comparison_index], similarity_matrix)
        if similarity_ratio > 0.2:
            similarity_flag = 1

    if (similarity_flag == 0) and (transport_all_contents[text_index] not in unique_articles):
        #текст потенциально подходит. проверим на наличие перекрестных тем
        #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
        exclude_count = 0
        tokens = prepare_text(transport_all_contents[text_index])
        for word in tokens:
            if word in exclude_list:
                exclude_count += 1
        if exclude_count == 0:
            unique_articles.append(transport_all_contents[text_index])
            unique_urls.append(transport_all_urls[text_index])
            unique_titles.append(transport_all_titles[text_index])
        else:
            pass

        #comparison_index += 1
    #text_index += 1
                
        
        

In [ ]:
def add_hyperlink(paragraph, url, text):
    """
    A function that places a hyperlink within a paragraph object.
    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

In [ ]:
#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Новости транспорта'#'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(unique_articles)):
    #try:
        header = unique_titles[i]
        document.add_heading(header, level = 1)
        content = unique_articles[i]
        document.add_paragraph(content)
        url = unique_urls[i]
        p=document.add_paragraph()
        #p = document.add_paragraph()
        url = add_hyperlink(p, url, 'Источник')
#         run = document.add_paragraph().add_run()
#         font = run.font
#         font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
#         p=document.add_paragraph()
#         url = unique_urls[i]
#         document.add_paragraph(url)
        
    #except:
        #pass

#doc_name = str(request_topic) + '.docx' 
doc_name = 'transport_' + str(current_date) + '.docx'
document.save(doc_name)

In [ ]:
len(unique_articles)

In [ ]:
word2vec_model300.most_similar('transport')

In [ ]:
    #формируем документ из коротких саммари новостей
    document = Document()
    doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
    doc_header = 'Тематика новостей: ' + str(request_topic)
    document.add_heading(doc_header)
    document.add_heading(doc_datestamp, level = 1)
    for i in range (0, len(all_articles['articles'])):
        header = all_articles['articles'][i]['title']
        document.add_heading(header, level = 1)
        try:
            content = summarize(text = all_articles['articles'][i]['content'], word_count= 100)
        except:
            content = all_articles['articles'][i]['content']
        document.add_paragraph(content)
        url = all_articles['articles'][i]['url']
        document.add_paragraph(url)

    #doc_name = str(request_topic) + '.docx' 
    doc_name = str(request_topic) + '_summary100.docx'
    document.save(doc_name)

### запрос по всей базе

In [ ]:
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')
request_topic = 'china'

# получение данных по теме
all_articles = newsapi.get_everything(q=request_topic,
                                      from_param=previous_day,
                                      to=current_date,
                                      language='en',
                                      sort_by='relevancy',
                                      page_size=10)

#Формирование служебных переменных
count = 0
error_link = [] #массив с битыми ссылками


#замеряем время
start_time = time.time()
print('--- Выгружаются новости... ---')

#попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
for i in range(0, len(all_articles['articles'])):
    url  = all_articles['articles'][i]['url']
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body_fulltext.append(text) 
        all_articles['articles'][i]['content'] = text 
    except:
        error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
        all_articles['articles'][i]['content'] = all_articles['articles'][i]['content'][0:258]#если статью не удалось выгрузить, поле COntent будем заполнять 259 знаками из самого описания
        pass
    count += 1
    if count % 5 == 0:
        print(count)

print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))

#сохранение новостей в базу

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)
    print('Выгрузка новостей сохранена в файл')

#формируем документ из коротких саммари новостей
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(all_articles['articles'])):
    header = all_articles['articles'][i]['title']
    document.add_heading(header, level = 1)
    try:
        content = summarize(text = all_articles['articles'][i]['content'], word_count= 100)
    except:
        content = all_articles['articles'][i]['content']
    document.add_paragraph(content)
    url = all_articles['articles'][i]['url']
    document.add_paragraph(url)

#doc_name = str(request_topic) + '.docx' 
doc_name = str(request_topic) + '_summary100.docx'
document.save(doc_name)

### Запрос по конкретной стране

In [ ]:
request_topic = 'transportation'

# получение данных по теме
top_headlines = newsapi.get_top_headlines(q=request_topic,
                                          #category='business',
                                          language='en',
                                          #country='cn')
                                         )

top_headlines

In [ ]:
top_headlines = newsapi.get_top_headlines(q=request_topic,
                                          #category='business',
                                          language='en',
                                          country='us')
top_headlines

In [ ]:
top_headlines

In [ ]:

request_topic = 'china land reform'

# получение данных по теме
top_headlines = newsapi.get_top_headlines(q=request_topic,
                                          category='business',
                                          language='en',
                                          pageSize = 10,
                                          country='cn')

#Формирование служебных переменных
count = 0
error_link = [] #массив с битыми ссылками


#замеряем время
start_time = time.time()
print('--- Выгружаются новости... ---')

#попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
for i in range(0, len(all_articles['articles'])):
    url  = all_articles['articles'][i]['url']
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body_fulltext.append(text) 
        all_articles['articles'][i]['content'] = text 
    except:
        error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
        all_articles['articles'][i]['content'] = all_articles['articles'][i]['content'][0:258]#если статью не удалось выгрузить, поле COntent будем заполнять 259 знаками из самого описания
        pass
    count += 1
    if count % 5 == 0:
        print(count)

print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))

#сохранение новостей в базу

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)
    print('Выгрузка новостей сохранена в файл')

#формируем документ из коротких саммари новостей
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(all_articles['articles'])):
    header = all_articles['articles'][i]['title']
    document.add_heading(header, level = 1)
    try:
        content = summarize(text = all_articles['articles'][i]['content'], word_count= 100)
    except:
        content = all_articles['articles'][i]['content']
    document.add_paragraph(content)
    url = all_articles['articles'][i]['url']
    document.add_paragraph(url)

#doc_name = str(request_topic) + '.docx' 
doc_name = str(request_topic) + '_summary100.docx'
document.save(doc_name)

In [ ]:
#Формирование массива ссылок из выдачи Api
count = 0
error_link = [] #массив с битыми ссылками


#замеряем время
start_time = time.time()
print('--- Выгружаются новости... ---')

#сохраним копию выгрузки с коротким содержанием новостей
all_articles_short = all_articles.copy()

#попытаемся скачать текст новости из первоисточника, если не получится - оставим краткое описание
for i in range(0, len(all_articles['articles'])):
    url  = all_articles['articles'][i]['url']
    gnews_links.append(url)
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body_fulltext.append(text) 
        all_articles['articles'][i]['content'] = text 
    except:
        error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
        all_articles['articles'][i]['content'] = all_articles['articles'][i]['content'][0:258]#если статью не удалось выгрузить, поле COntent будем заполнять 259 знаками из самого описания
        pass
    count += 1
    if count % 5 == 0:
        print(count)
    
print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))

#сохранение новостей в базу

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + str(request_topic) + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)
    print('Выгрузка новостей сохранена в файл')
    
document = Document()
document.add_heading(request_topic)
for i in range (0, len(all_articles['articles'])):
    header = all_articles['articles'][i]['title']
    document.add_heading(header, level = 1)
    content = summarize(text = all_articles['articles'][i]['content'], word_count= 100)
    #all_articles['articles'][i]['content']
    document.add_paragraph(content)
    url = all_articles['articles'][i]['url']
    document.add_paragraph(url)

#doc_name = str(request_topic) + '.docx' 
doc_name = str(request_topic) + '_summary100.docx'
document.save(doc_name)

In [ ]:
document = Document()
doc_datestamp = 'Период: ' + str(previous_day) + ' - ' + str(current_date)
doc_header = 'Тематика новостей: ' + str(request_topic)
document.add_heading(doc_header)
document.add_heading(doc_datestamp, level = 1)
for i in range (0, len(all_articles['articles'])):
    header = all_articles['articles'][i]['title']
    document.add_heading(header, level = 1)
    content = summarize(text = all_articles['articles'][i]['content'], word_count= 100)
    #all_articles['articles'][i]['content']
    document.add_paragraph(content)
    url = all_articles['articles'][i]['url']
    document.add_paragraph(url)

#doc_name = str(request_topic) + '.docx' 
doc_name = str(request_topic) + '_summary100.docx'
document.save(doc_name)


### Блок выгрузки из ГУГЛа и предобработки новостей

In [ ]:
transport_topiclist_rus = ['Avtovaz', 'Lada', 'Kia', 'Hyundai ', 'Avtodizel', 'GAZ', 'Gazel', 'KAMAZ']

In [ ]:
#функция предобработки русскоязычных текстов
from nltk.corpus import stopwords
stop_rus = stopwords.words('russian')

def prepare_text_russian(text, stop = stop_rus, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return    words#' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

In [84]:
#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return    words#' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

In [9]:


def googleparser (topic, depth):
    '''
    topic - тема новостного запроса
    depth - число опрашиваемых страниц поисковой выдачи
    
    Возвращает два списка: с текстами новостей и с проблемными ссылками 
    
    '''
    # 1. формирование массива ссылок
    gnews_links = []
    gnews=[]
    googlenews.search(topic)
    
    start_time = time.time()
    
    print('--- Формируется массив ссылок... ---')

    for i in range(1,depth):
        googlenews.clear()
        googlenews.getpage(i)
        for j in range(0, len(googlenews.gettext())):
            gnews.append(googlenews.gettext()[j])
            gnews_links.append(googlenews.getlinks()[j])

    print("--- На формирование массива затрачено %s секунд ---" % (time.time() - start_time))             
    print('--- Завершено. Получено %s ссылок ---' % len(gnews_links))   
    
    # 2. выгрузка новостей и формирование массива текстов
    
    body = []
    count = 0
    error_link = [] #массив с битыми ссылками

    #замеряем время
    start_time = time.time()

    print('--- Выгружаются новости... ---')

    for url in gnews_links:
        try:
            html = requests.get(url).text
            text = fulltext(html)
            body.append(text)
        except:
            error_link.append(gnews_links[count]) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
            pass
        count += 1
        if count % 5 == 0:
            print(count)

    print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))             
    return body, error_link

In [ ]:
googlenews = GoogleNews('ru')
topic = transport_topiclist_rus[0]
googlenews.search(topic)



In [ ]:
googlenews.result()

In [ ]:
# Будем запрашивать новости из гугла на русском языке транслитом (т.к. модуль почему-то не работает с выгрузками на англ)
googlenews = GoogleNews('ru')
all_articles = []
for topic in transport_topiclist_rus:
    googlenews.clear()
    googlenews.search(topic)
    for item in googlenews.result():
        all_articles.append(item)


In [ ]:
#сформируем массивы с данными о заглавиях, ссылках и содержании новостей
all_contents = []
all_titles = []
all_urls = []

for article_number in range(0,len(all_articles)):
    url  = all_articles[article_number]['link']
    try:
        html = requests.get(url).text
        text = fulltext(html, language = 'ru')
    except:
        text = all_articles[article_number]['desc']
    if text != '':
        all_contents.append(text)
        all_titles.append(all_articles[article_number]['title'])
        all_urls.append(url)   
    else:
        pass

In [ ]:
nltk.download()


In [ ]:
prepare_text(all_contents[0])

In [ ]:
# проверяем можно ли записать вместо текстов саммари
for i in range (0, len(all_contents)):
    try:
        summary = summarize(text = all_contents[i], word_count = 100)
        if len(summary) > 0:
            all_contents[i] = summary
    except:
        pass

In [ ]:
tokenized_list = []
for doc in all_contents:
    try:
        tokens = prepare_text(doc)
        tokenized_list.append(tokens)
        error_index += 1
    except:
        error_list.append(error_index)
        error_index += 1
#отправляем это хозяйство в словарь и в корпус
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora import Dictionary
from gensim.models import Word2Vec, WordEmbeddingSimilarityIndex
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix
from gensim.models.keyedvectors import WordEmbeddingSimilarityIndex



In [ ]:
#создаем матрицу схожести на основе модели Word2vec
#здесь считается косинусная близость для всех пар векторов (слов)
termsim_index = WordEmbeddingSimilarityIndex(word2vec_model300)
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, mydict)

In [ ]:
# конвертируем тексты новостей в bagofwords и создаем массив из списков bow
bag_of_words = []
for text in tokenized_list:
    bag_of_words.append(mydict.doc2bow(text))

In [ ]:
# считаем косинусную близость между всеми документами
docsim_index = SoftCosineSimilarity(bag_of_words, similarity_matrix)

### Тест выгрузок по одной теме за раз

In [102]:
transport_topic_list = ['maritime transport', 'port freight', 'container ship', 'bulk carrier', 'tanker', 'TEU', 'auto industry', 'refrigerated ship', 'reefer', 'supertanker', 'cargo ship']

In [93]:
# слова-исключения, по которым будут отфильтровываться статьи
exclude_list = ['macbook','laptop','ultrabook','server', 'api', 'usb', 'installer', 'ubuntu', 'ip', 'lan','javascript', 'captcha', 'browser','internet', 'usb-c', 'nintendo', 'docker', 'desktop', 'spacex', 'windows']  

In [165]:
# agro_topic_list = ['+china AND (+red +line)', '+china +((agriculture | food) AND subsidy)', '+china +gmo', '+china +(food AND reserve)', '+china +(agriculture AND policy)', '+india AND +fertilizer', '+monsoon | +(el AND nino) | (+USA  +drought)', 'brazil AND (inflation | real)', 
#              '+argentina +(export AND tariff)',  '+EU | +Europe +(food AND quota)']
agro_topic_list = ['china +red +line', 'china agriculture food subsidy', 'china +gmo', 'china food reserve', 'china agriculture policy', 'india fertilizer', 'monsoon el nino drought', 'brazil inflation real', 
              'argentina export tariff',  'EU food quota']

In [124]:
itt_topic_list = ['semiconductors market', '+software +market +volume', 'telecommunications market volume', 'iaas | paas | saas', '+it +market', 'computer sales', 'IT coronavirus']

In [117]:
def prepare_news_list_google(topiclist, topicname):
    articles_base = {}
    for topic in topiclist:
        # выгрузка статей по одному запросу
        all_articles = []
        googlenews.clear()
        googlenews.search(topic)
        for item in googlenews.result()[0:5]:
            all_articles.append(item)
            
        #dict of topics and data for each topic

        articles_base[topic] = all_articles
        
    # формируем границы периода времени, за которое будем запрашивать новости
    current_date = datetime.today()
    previous_day = current_date - timedelta(days = 1)
    current_date = current_date.strftime('%Y-%m-%d')
    previous_day = previous_day.strftime('%Y-%m-%d')

    #формируем документ из коротких саммари новостей отфильтрованных на различность.
    document = Document()

    doc_header = 'Глобальные новости' + '\n' + 'Период: ' + str(previous_day) + ' - ' + str(current_date)
    document.add_heading(doc_header)
    
    for topic in articles_base:
        count = 1
        header = 'Новости по запросу: ' + str(topic)
        document.add_heading(header, level = 2)
        for item in articles_base[topic]:
            tokens = prepare_text(item['title'])
            raise_flag_excludeword = 0
            for token in tokens:
                if token in exclude_list:
                    raise_flag_excludeword = 1
            if raise_flag_excludeword == 0:         
                url = item['link']
                p = document.add_paragraph()
                link_text = str(count) + '. ' + item['title'] 
                url = add_hyperlink(p, url, link_text)
                count += 1

    #doc_name = str(request_topic) + '.docx' 
    doc_name = str(topicname) + '_' + str(current_date) + '.docx'
    document.save(doc_name)  

In [118]:
def prepare_news_list_newsapi(topiclist, topicname):
    articles_base = {}
    for topic in topiclist:

        
        # формируем границы периода времени, за которое будем запрашивать новости
        current_date = datetime.today()
        previous_day = current_date - timedelta(days = 1)
        current_date = current_date.strftime('%Y-%m-%d')
        previous_day = previous_day.strftime('%Y-%m-%d')

        # получение данных по теме
        all_articles = newsapi.get_everything(q=topic,
                                              from_param=previous_day,
                                              to=current_date,
                                              language='en',
                                              sort_by='relevancy',
                                              page_size=10)


        articles_base[topic] = all_articles['articles']

    #формируем документ из коротких саммари новостей отфильтрованных на различность.
    document = Document()

    doc_header = 'Глобальные новости' + '\n' + 'Период: ' + str(previous_day) + ' - ' + str(current_date)
    document.add_heading(doc_header)

    for topic in articles_base:
        count = 1
        header = 'Новости по запросу: ' + str(topic)
        document.add_heading(header, level = 2)
        for item in articles_base[topic]:
            tokens = prepare_text(item['title'])
            raise_flag_excludeword = 0
            for token in tokens:
                if token in exclude_list:
                    raise_flag_excludeword = 1
            if raise_flag_excludeword == 0:         
                url = item['url']
                p = document.add_paragraph()
                link_text = str(count) + '. ' + item['title'] 
                url = add_hyperlink(p, url, link_text)
                count += 1

    #doc_name = str(request_topic) + '.docx' 
    doc_name = str(topicname) + '_' + str(current_date) + '.docx'
    document.save(doc_name)  

In [166]:
def prepare_news_list_aggregate(topiclist, topicname):
    # формируем границы периода времени, за которое будем запрашивать новости
    current_date = datetime.today()
    previous_day = current_date - timedelta(days = 1)
    current_date = current_date.strftime('%Y-%m-%d')
    previous_day = previous_day.strftime('%Y-%m-%d')

    articles_base_google = {}
    articles_base_newsapi = {}
    articles_base = {}
    for topic in topiclist:
        # выгрузка статей по одному запросу
        all_articles = []
        googlenews.clear()
        googlenews.search(topic)
        for item in googlenews.result()[0:5]:
            all_articles.append(item)

        articles_base_google[topic] = all_articles



        # получение данных по теме
        all_articles = newsapi.get_everything(q=topic,
                                              from_param=previous_day,
                                              to=current_date,
                                              language='en',
                                              sort_by='relevancy',
                                              page_size=5)


        articles_base_newsapi[topic] = all_articles['articles']

        articles_base[topic] = articles_base_google[topic] + articles_base_newsapi[topic]

    #формируем документ из коротких саммари новостей отфильтрованных на различность.
    document = Document()

    doc_header = 'Глобальные новости' + '\n' + 'Период: ' + str(previous_day) + ' - ' + str(current_date)
    document.add_heading(doc_header)

    for topic in articles_base:
        count = 1
        header = 'Новости по запросу: ' + str(topic)
        document.add_heading(header, level = 2)
        for item in articles_base[topic]:
            tokens = prepare_text(item['title'])
            raise_flag_excludeword = 0
            for token in tokens:
                if token in exclude_list:
                    raise_flag_excludeword = 1
            if raise_flag_excludeword == 0: 
                try:
                    url = item['url']
                except:
                    url = item['link']
                p = document.add_paragraph()
                link_text = str(count) + '. ' + item['title'] 
                url = add_hyperlink(p, url, link_text)
                count += 1

    doc_name = 'aggregate_' + str(topicname) + '_' + str(current_date) + '.docx'
    document.save(doc_name) 

In [99]:
prepare_news_list_newsapi(topic_list)

In [125]:
prepare_news_list_google(itt_topic_list, 'ITT')

In [126]:
prepare_news_list_google(topic_list, 'Transport')

In [167]:
prepare_news_list_aggregate(agro_topic_list, 'Agriculture')

In [162]:
prepare_news_list_aggregate(topic_list, 'Transport')

In [168]:
prepare_news_list_aggregate(itt_topic_list, 'ITT')

In [2]:
topiclist = ['maritime transport', 'port freight', 'container ship', 'bulk carrier', 'tanker', 'TEU', 'auto industry', 'refrigerated ship', 'reefer', 'supertanker', 'cargo ship']
topicname = 'Trans'
# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

articles_base_google = {}
articles_base_newsapi = {}
articles_base = {}
for topic in topiclist:
    # выгрузка статей по одному запросу
    all_articles = []
    googlenews.clear()
    googlenews.search(topic)
    for item in googlenews.result()[0:5]:
        all_articles.append(item)

    articles_base_google[topic] = all_articles



    # получение данных по теме
    all_articles = newsapi.get_everything(q=topic,
                                          from_param=previous_day,
                                          to=current_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page_size=5)


    articles_base_newsapi[topic] = all_articles['articles']

    articles_base[topic] = articles_base_google[topic] + articles_base_newsapi[topic]

# #формируем документ из коротких саммари новостей отфильтрованных на различность.
# document = Document()

# doc_header = 'Глобальные новости' + '\n' + 'Период: ' + str(previous_day) + ' - ' + str(current_date)
# document.add_heading(doc_header)

# for topic in articles_base:
#     count = 1
#     header = 'Новости по запросу: ' + str(topic)
#     document.add_heading(header, level = 2)
#     for item in articles_base[topic]:
#         tokens = prepare_text(item['title'])
#         raise_flag_excludeword = 0
#         for token in tokens:
#             if token in exclude_list:
#                 raise_flag_excludeword = 1
#         if raise_flag_excludeword == 0: 
#             try:
#                 url = item['url']
#             except:
#                 url = item['link']
#             p = document.add_paragraph()
#             link_text = str(count) + '. ' + item['title'] 
#             url = add_hyperlink(p, url, link_text)
#             count += 1

# doc_name = 'aggregate_' + str(topicname) + '_' + str(current_date) + '.docx'
# document.save(doc_name) 

In [8]:
articles_df = pd.DataFrame.from_records(articles_base['maritime transport'])

In [9]:
articles_df.head()

,title,media,date,desc,link,img,source,author,description,url,urlToImage,publishedAt,content
0,"Maritime transport, logistics target N78 trill...",,,REPORTS from global port and marine operations...,https://www.vanguardngr.com/2020/05/maritime-t...,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maritime Transport Consulting Service Market 2...,,,Maritime Transport Consulting Service Market 2...,https://coleofduty.com/military-news/2020/05/0...,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eckerö Line and DFDS started cooperation,,,DFDS is one of Europe's largest maritime trans...,https://en.portnews.ru/news/295432/,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Body found on boat in Tauranga Harbour: Man ch...,,,The court charging document revealed the mansl...,https://www.nzherald.co.nz/nz/news/article.cfm...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Proposal To Trial Suburban Commuter Passenger ...,,,A Proposal from the Rail and Maritime Transpor...,https://www.scoop.co.nz/stories/PO2005/S00079/...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
articles_base['maritime transport']

[{'title': 'Maritime transport, logistics target N78 trillion revenue by 2023',
  'media': '',
  'date': '',
  'desc': 'REPORTS from global port and marine operations, indicate that global maritime freight transportation revenue is expected to grow from $166 billion last year to\xa0...',
  'link': 'https://www.vanguardngr.com/2020/05/maritime-transport-logistics-target-n78-trillion-revenue-by-2023/',
  'img': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXf-_FR_Yhl6juYrdIOC-_iUrt_pjeixbD12Z0ICfWlDY52kkR_wm60TbPNWI3ezeNDnkU4CyF&s'},
 {'title': 'Maritime Transport Consulting Service Market 2020 | Growth ...',
  'media': '',
  'date': '',
  'desc': 'Maritime Transport Consulting Service Market 2020 | Growth Drivers, Challenges, Trends, Market Dynamics and Forecast to 2026. Maritime Transport\xa0...',
  'link': 'https://coleofduty.com/military-news/2020/05/06/maritime-transport-consulting-service-market-2020-growth-drivers-challenges-trends-market-dynamics-and-forecast-to-2026/',

In [134]:
#сформируем массивы с данными о заглавиях, ссылках и содержании новостей
all_contents = []
all_titles = []
all_urls = []


for article_number in range(0,len(all_articles)):
    url  = all_articles[article_number]['link']
    try:
        html = requests.get(url).text
        text = fulltext(html, language = 'ru')
    except:
        text = all_articles[article_number]['desc']
    if text != '':
        all_contents.append(text)
        all_titles.append(all_articles[article_number]['title'])
        all_urls.append(url)   
    else:
        pass

SyntaxError: invalid syntax (<ipython-input-134-f1f738685f11>, line 1)

In [ ]:
text_index = 0
comparison_index = 0
unique_articles = []
unique_urls = []
unique_titles = []

for text_index in range(0, len(bag_of_words)):
    query = tokenized_list[text_index]
    if query != []:
        similarity_flag = 0
        sims = docsim_index[mydict.doc2bow(query)]
        for comparison_index in range(text_index + 1, len(bag_of_words)):
            similarity_ratio = sims[comparison_index]
            if similarity_ratio > 0.2:
                similarity_flag = 1

        if (similarity_flag == 0) and (all_contents[text_index] not in unique_articles):
            #текст потенциально подходит. проверим на наличие перекрестных тем
            #будем считать текст не по теме, если в нем встречается 2 и более посторонних тематических слов
            exclude_count = 0
            tokens = prepare_text(all_contents[text_index])
            for word in tokens:
                if word in exclude_list:
                    exclude_count += 1
            if exclude_count == 0:
                unique_articles.append(all_contents[text_index])
                unique_urls.append(all_urls[text_index])
                unique_titles.append(all_titles[text_index])
            else:
                pass
    else:
        pass



In [86]:
# формируем границы периода времени, за которое будем запрашивать новости
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

#формируем документ из коротких саммари новостей отфильтрованных на различность.
document = Document()

doc_header = 'Новости транспорта' + '\n' + 'Период: ' + str(previous_day) + ' - ' + str(current_date)
document.add_heading(doc_header)

for topic in articles_base:
    count = 1
    header = 'Новости по запросу: ' + str(topic)
    document.add_heading(header, level = 2)
    for item in articles_base[topic]:
        tokens = prepare_text(item['title'])
        raise_flag_excludeword = 0
        for token in tokens:
            if token in exclude_list:
                raise_flag_excludeword = 1
        if raise_flag_excludeword == 0:         
            url = item['url']
            p = document.add_paragraph()
            link_text = str(count) + '. ' + item['title'] 
            url = add_hyperlink(p, url, link_text)
            count += 1



#doc_name = str(request_topic) + '.docx' 
doc_name = 'experimental_transport_' + str(current_date) + '.docx'
document.save(doc_name)  

In [ ]:
#body, errors = gp.googleparser('China economy',20)
body, errors = googleparser('Agriculture',10)

In [ ]:
with open('unlabeled_body_20200207.txt', 'w',encoding="utf-8") as f:
    i=0
    for item in body:
        f.write('Story %s\n' %(i))
        f.write(item.replace('\n\n','\n'))
        f.write('\n\n')
        i += 1

### запрос по api

In [ ]:
## запрос по api
import requests
import json
from newsapi import NewsApiClient

In [ ]:
url = ('https://newsapi.org/v2/everything?'
      'q=maritime transport&'
      'from=2020-03-27&'
      'sortBy=popularity&'
      'apiKey=8a38d185855d4fbda8c437e94149807d')

response = requests.get(url)

In [ ]:
#сохранение выгрузки новостей в базу
json_data_agro = json.loads(response.text)
json_data_agro['articles']

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + 'agro' + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(json_data_agro, f, ensure_ascii=False, indent=4)

In [ ]:
#сохранение выгрузки новостей в базу
json_data_china = json.loads(response.text)
json_data_china['articles']

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + 'china' + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(json_data_china, f, ensure_ascii=False, indent=4)

In [ ]:
###выгрузка корпуса новостей, их сохранение и предобработка

In [ ]:
newsapi = NewsApiClient(api_key='8a38d185855d4fbda8c437e94149807d')
current_date = datetime.today()
previous_day = current_date - timedelta(days = 1)
current_date = current_date.strftime('%Y-%m-%d')
previous_day = previous_day.strftime('%Y-%m-%d')

In [ ]:
# получение данных по Китаю
all_articles = newsapi.get_everything(q='maritime transport',
                                      from_param=previous_day,
                                      to=current_date,
                                      language='en',
                                      sort_by='relevancy',
                                      page_size=10)

#Формирование массива ссылок из выдачи Api
news_links = []
body_fulltext = []
count = 0
error_link = [] #массив с битыми ссылками
gnews_links = []
gnews=[]

#замеряем время
start_time = time.time()
print('--- Выгружаются новости... ---')

for i in range(0, len(all_articles['articles'])):
    url  = all_articles['articles'][i]['url']
    gnews_links.append(url)
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body_fulltext.append(text)
        all_articles['articles'][i]['content'] = text
    except:
        error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
        all_articles['articles'][i]['content'] = all_articles['articles'][i]['content'][0:258]#если статью не удалось выгрузить, поле COntent будем заполнять 259 знаками из самого описания
        pass
    count += 1
    if count % 5 == 0:
        print(count)
    
print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))

#сохранение новостей в базу

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + 'maritime' + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)
    print('Выгрузка новостей сохранена в файл')

In [ ]:
# получение данных по миру
all_articles = newsapi.get_everything(q='world economy',
                                      from_param=current_date,
                                      to=current_date,
                                      language='en',
                                      sort_by='relevancy',
                                      page_size=100)

#Формирование массива ссылок из выдачи Api
news_links = []
body_fulltext = []
count = 0
error_link = [] #массив с битыми ссылками
gnews_links = []
gnews=[]

#замеряем время
start_time = time.time()
print('--- Выгружаются новости... ---')

for i in range(0, len(all_articles['articles'])):
    url  = all_articles['articles'][i]['url']
    gnews_links.append(url)
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body_fulltext.append(text)
        all_articles['articles'][i]['content'] = text
    except:
        error_link.append(url) #иногда попадаются проблемные ссылки. Здесь мы будем сохранять их 
        all_articles['articles'][i]['content'] = all_articles['articles'][i]['content'][0:258]#если статью не удалось выгрузить, поле COntent будем заполнять 259 знаками из самого описания
        pass
    count += 1
    if count % 5 == 0:
        print(count)
    
print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))

#сохранение агро новостей в базу

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + 'world' + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)
    print('Выгрузка новостей сохранена в файл')

In [ ]:
body_fulltext

In [ ]:
#сохранение китайских новостей в базу

save_path = 'C:/Users/Egran/YandexDisk/Work/news_project/news_database/'
new_filename = save_path + 'china' + '_' + str(time.gmtime().tm_year) + '_' + str(time.gmtime().tm_mon) + '_' + str(time.gmtime().tm_mday) + '.txt'
with open(new_filename, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=4)

In [ ]:
len(body_fulltext)

In [ ]:
i=0
for article in json_data_china['articles']:
    print('article', i)
    print(article['description'])
    i += 1
    print('\n')

In [ ]:
i = 0
body = []
for article in json_data_ncov['articles']:
    try:
        print('article', i)
        print(article['content'][0:258])
        body.append(article['content'][0:258])
        
        
    except:
        pass
    print('\n')
    i += 1



In [ ]:
for article in json_data_ncov['articles']:
    try:
        print('article', i)
        print(article['content'][0:258])
        body.append(article['content'][0:258])
        
        
    except:
        pass
    print('\n')


In [ ]:
for company in companies:
    try:
        body, errors = gp.googleparser(company,30)
        with open(company + '.txt', 'w',encoding="utf-8") as f:
            i=0
            for item in body:
                f.write('Story %s\n' %(i))
                f.write(item.replace('\n\n','\n'))
                f.write('\n\n')
                i += 1
    except Exception as e:
        print(str(e))

In [ ]:
#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in body_fulltext:
    prepared_texts.append(prepare_text(item))

In [ ]:
#Блок выгрузки линков и заголовков новостей из гугла
#новости выгружаются по заданной теме, глубина выгрузки - 10 страниц

topic = 'China'
gnews_links = []
gnews=[]
googlenews.search(topic)

#замеряем время
start_time = time.time()

print('--- Формируется массив ссылок... ---')

for i in range(1,2):
    googlenews.clear()
    googlenews.getpage(i)
    for j in range(0, len(googlenews.gettext())):
        gnews.append(googlenews.gettext()[j])
        gnews_links.append(googlenews.getlinks()[j])

print("--- На формирование массива затрачено %s секунд ---" % (time.time() - start_time))             
print('--- Заврешено. Выгружено %s статей ---' % len(gnews_links))    

In [ ]:
#блок выгрузки новостей и формирования массива текстов
body = []
count = 0
error_position = [] #массив с номерами битых ссылок

#замеряем время
start_time = time.time()

print('--- Выгружаются новости... ---')

for url in gnews_links:
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body.append(text)
    except:
        error_position.append(count) #иногда попадаются битые ссылки. Здесь мы будем сохранять их позиции
        pass
    count += 1
    
print("--- Завершено. На выгрузку затрачено %s секунд ---" % (time.time() - start_time))             


### Темы

In [ ]:
# Create the Inputs of LDA model: Dictionary and Corpus 
#(doc2bow array of id's of words and their frequency)

dct = corpora.Dictionary(prepared_texts)
corpus = [dct.doc2bow(item) for item in prepared_texts]
#dct.save('news.dict')

In [ ]:
corpora.MmCorpus.serialize('news.model',corpus)
#print(corpus)

In [ ]:
# Load trained model and update the model on new data

lda_model = LdaMulticore.load(datapath("C:/Users/Egran/YandexDisk/Work/news_project/lda_model"))
lda_model.update(corpus)
lda_model.save('lda_model')

In [ ]:
#Train new LDA model

lda_daily_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         random_state=100,
                         num_topics=9,
                         passes=50,
                         chunksize=100,
                         batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

# save the model
#lda__daily_model.save('lda_model')

In [ ]:
# See the topics
#lda_model.print_topics(-1)
lda_daily_model.show_topics(num_topics=9, num_words=10, formatted=False)

In [ ]:
lda_daily_model.top_topics(texts=prepared_texts)

In [ ]:
lda_daily_model.log_perplexity(corpus)

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_daily_model, texts=prepared_texts, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### Визуализация выводов тем

In [ ]:
# Вывод тем по обученной LDA модели
# 1. Wordcloud of Top N words in each topic


cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=5,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_daily_model.show_topics(formatted=False)
#topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(3, 3, figsize=(20,20), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
# tf-idf на векторах
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# создание случайной выборки
sampling_tfidf = random.choices(corpus_tfidf, k=30)

index = similarities.MatrixSimilarity(sampling_tfidf)
sims = index[sampling_tfidf]

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(data=sims, cmap = 'Spectral').set(xticklabels=[], yticklabels=[])
plt.title("Матрица близости")
plt.show()

In [ ]:
# more on visuals
# https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/

In [ ]:
#https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c

In [ ]:
print(dct.token2id)

### Интерактивная визуализация

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_daily_model, corpus, dct, mds='mmds')
vis

### Саммари

In [ ]:
#Блок предобработки текста для рейтера (он аналогичен блоку выше, но возвращает строки, а не отдельные слова)

stop_words = stopwords.words('english')
stop_words = stop_words + ['china', 'chinese','china\'s','china’s'] #в стоп список включаем слова, которые являются перекрестными в разных темах
lemmatizator = WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def reuter_prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст 

    parameters:  
        stop: список из стоп-слов
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in words if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return ' '.join(words)#words  #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста. 

In [ ]:

def china_summary_reuters(depth):
    """
    Подготовка резюме по основным новостям по Китаю
    depth - количество заголовков для просмотра на странице с последними новостями по Китаю
    """
      
    #Выгружаем все свежие заголовки про Китай
    r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
    #r1 = requests.get('https://www.reuters.com/search/news?blob=metals')
    coverpage =  r1.content
    soup = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news_r = soup.find_all('h2', class_='FeedItemHeadline_headline')

    #Опционально можно вывести отдельно все заголовки:
#     for i in range(0, len(coverpage_news_r)):
#         print(coverpage_news_r[i].get_text())
#         print('///')

    # Опционально можно парсить все статьи по существующим заголовкам без ограничения глубины
    #number_of_articles = len(coverpage_news_r)
    
    # Empty lists for content, links and titles
    news_contents = []
    list_links = []
    list_titles = []

    #замеряем время
    start_time = time.time()
    print('---Формируется подборка статей---')

    #формируем массив статей
    for n in range(0, depth):


        # Getting the link of the article
        link = coverpage_news_r[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news_r[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('div', class_='StandardArticleBody_body')
        try:
            x = body[0].find_all('p')
        except:
            pass
        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(x)):
            paragraph = x[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        news_contents.append(final_article)
    print("--- Подборка сформирована. Затрачено %s секунд ---" % (time.time() - start_time))
    
    #Проходим по массиву выгруженных новостей и выполняем предобработку
    print('---Предобработка---')
    prepared_texts = []
    for item in news_contents:
        prepared_texts.append(reuter_prepare_text(item))
    print('---Предобработка выполнена---')
    print('///')
    #задаем ключевые слова по темам
    topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR','trade']

    # отбор новостей по ключевым словам и суммаризация
    i = 0
    for item in prepared_texts:
        counter = 0
        for word in item.split(' '):
            if word in topics_econ:
                counter += 1
        if counter > 2:
            print('Economic story %s' % i)
            print(summarize(item, word_count=60))
            print('Ссылка на статью:', list_links[i])
            print('///')
            i += 1


## Старые эксперименты

In [ ]:
#Выгружаем все свежие заголовки про Китай
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
#r1 = requests.get('https://www.reuters.com/search/news?blob=metals')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('h2', class_='FeedItemHeadline_headline')

#Выводим заголовки
for i in range(0, len(coverpage_news_r)):
    print(coverpage_news_r[i].get_text())
    print('///')

In [ ]:
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()




In [ ]:
#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))


In [ ]:
prepared_texts[0]

In [ ]:
#задаем ключевые слова по темам
topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR','trade']
topics_metals = ['metal', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']
print(summarize(prepared_texts[0], word_count=60))


In [ ]:
# отбор новостей по ключевым словам
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_econ:
            counter += 1
    if counter > 2:
        print('Economic story %s', i)
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1


### Блок металлов

In [ ]:
#Выгружаем все свежие заголовки про металлы из рейтерс
# s = requests.session()
# s.cookies.clear()

#r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
#https://www.reuters.com/news/archive/metals-news
r1 = requests.get('https://www.reuters.com/search/news?blob=Metals&sortBy=date&dateRange=all')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('h3', class_='search-result-title')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(coverpage_news_r[i].get_text())
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = 'https://www.reuters.com'+coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 3 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


In [ ]:
#Парсинг статей про металлы из архива
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('https://www.reuters.com/news/archive/metals-news')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('div', class_='story-content')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(re.sub('\t','',coverpage_news_r[i].get_text()))
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = 'https://www.reuters.com'+coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 1 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


In [ ]:
#Парсинг статей про металлы с сайта Kitco
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('http://www.kitcometals.com/news/')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('td', class_='text')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(re.sub('\t','',coverpage_news_r[i].get_text()))    #таким образом убираем тупые пробелы перед заглавием
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('article', itemprop='articleBody')
    try:
        x = body[0].find_all('p')
    except:
        pass

    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 1 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item
